In [1]:
import numpy as np

# A four months American put with strike $50 on an asset with spot price $48 paying dividends
# continuously at rate 1% is worth $4.08. The risk free interest rates are constant at 2%
# parameters
S0 = 48
K = 50
r = 0.02
q = 0.01
T = 4/12
N = 10
tol = 1e-6

# Identify the number of time steps Nfixed in the trinomial tree pricer which gives the value
# of a four months put option within 10^-6 tolerance:
# Start with 10 time steps for the given option and double the time steps until two consecutive
# approximations are within tolerance of each other. The finest tree considered corresponds to
# Nfixed time steps.

def european_put_trinomial(S, K, T, r, q, sigma, N):
    dt = T / N
    u = np.exp(sigma * np.sqrt(3*dt))
    d = 1 / u
    pu = 1/6 + (r-q-sigma**2*0.5) * np.sqrt(dt/(12*sigma**2))
    pm = 2/3
    pd = 1/6 - (r-q-sigma**2*0.5) * np.sqrt(dt/(12*sigma**2))

    # Build the binomial tree
    option_tree = np.zeros((N + 1, 2*N + 1))
    for j in range(2*N + 1):
        option_tree[N][j] = max(0, K - S * (u ** (N - j)))

    # Backward induction
    for i in range(N - 1, -1, -1):
        for j in range(2*i+1):
            option_tree[i][j] = np.exp(-r * dt) * (pu * option_tree[i + 1][j] + pm * option_tree[i + 1][j + 1] + pd * option_tree[i+1][j+2])

    return option_tree[0][0]


# american option
def american_put_trinomial(S, K, T, r, q, sigma, N):
    dt = T / N
    u = np.exp(sigma * np.sqrt(3*dt))
    d = 1 / u
    pu = 1/6 + (r-q-sigma**2*0.5) * np.sqrt(dt/(12*sigma**2))
    pm = 2/3
    pd = 1/6 - (r-q-sigma**2*0.5) * np.sqrt(dt/(12*sigma**2))

    # Build the binomial tree for the option values
    option_tree = np.zeros((N + 1, 2*N + 1))
    for j in range(2*N + 1):
        option_tree[N][j] = max(0, K - S * (u ** (N - j)))

    # Backward induction
    for i in range(N - 1, -1, -1):
        for j in range(2*i+1):
            intrinsic = K - S * (u ** (i - j))
            option_tree[i][j] = max(intrinsic, np.exp(-r * dt) * (pu * option_tree[i + 1][j] + pm * option_tree[i + 1][j + 1] + pd * option_tree[i+1][j+2]))

    return option_tree[0][0]

sigma = 0.272903
N = 10
prev = 0
while True:
    V = european_put_trinomial(S0, K, T, r, q, sigma, N)
    if abs(V - prev) < tol:
        break

    print('The number of time steps in the finest tree is', N)
    print('The option price is', V)

    prev = V
    N *= 2

# final iteration of the binomial tree pricer
N_fixed = N

print('The number of time steps in the finest tree is', N_fixed)
print('The option price is', V)


The number of time steps in the finest tree is 10
The option price is 4.097595928369878
The number of time steps in the finest tree is 20
The option price is 4.073699338935251
The number of time steps in the finest tree is 40
The option price is 4.048241976322065
The number of time steps in the finest tree is 80
The option price is 4.064270467759336
The number of time steps in the finest tree is 160
The option price is 4.059238832934539
The number of time steps in the finest tree is 320
The option price is 4.061931140533257
The number of time steps in the finest tree is 640
The option price is 4.061220764398057
The number of time steps in the finest tree is 1280
The option price is 4.061415049319836
The number of time steps in the finest tree is 2560
The option price is 4.061341016541253


KeyboardInterrupt: 

In [19]:
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')

S0 = 48
K = 50
r = 0.02
q = 0.01
T = 4/12
N = 10
tol = 1e-4

# american option
def american_put_trinomial(S, K, T, r, q, sigma, N):
    dt = T / N
    u = np.exp(sigma * np.sqrt(3*dt))
    d = 1 / u
    pu = 1/6 + (r-q-sigma**2*0.5) * np.sqrt(dt/(12*sigma**2))
    pm = 2/3
    pd = 1/6 - (r-q-sigma**2*0.5) * np.sqrt(dt/(12*sigma**2))

    # Build the binomial tree for the option values
    option_tree = np.zeros((N + 1, 2*N + 1))
    for j in range(2*N + 1):
        option_tree[N][j] = max(0, K - S * (u ** (N - j)))

    # Backward induction
    for i in range(N - 1, -1, -1):
        for j in range(2*i+1):
            intrinsic = K - S * (u ** (i - j))
            option_tree[i][j] = max(intrinsic, np.exp(-r * dt) * (pu * option_tree[i + 1][j] + pm * option_tree[i + 1][j + 1] + pd * option_tree[i+1][j+2]))

    return option_tree[0][0]

option_price = 4.08

sigma0 = 0.05  # Initial volatility
sigma1 = 1  # Initial volatility
prev = 0  # Previous option price

# Use the secant method with initial volatility approximations 5% volatility and 100% volatility, i.e., sigma0 = 0.05 and sigma1 = 1. Use tolerance 10−4
# . Report all the implied volatility approximations from the secant method recursion as well as the corresponding trinomial tree approximate
# values of the option

res = pd.DataFrame(columns=['sigma', 'option_price'])
N = 640
while abs(sigma1 - sigma0) > tol:
    V = american_put_trinomial(S0, K, T, r, q, sigma1, N)
    print(sigma1, V)
    res = res.append({'sigma': sigma, 'option_price': V}, ignore_index=True)
    sigma = (sigma1-sigma0)*(V-option_price)/(V-prev)
    sigma0 = sigma1
    sigma1 = sigma
    prev = V
    
print('The implied volatility is', sigma)
print('The option price is', V)
res

1 12.02821752222688
0.6277577398448639 7.980242271349651
0.35865708366798216 5.016553701759442
0.0850383600606191 2.1799971582245217
-0.18327727455384987 3.0208481103378224
0.33797548865655364 4.789302767730655
0.2090672927840086 3.392563691955533
-0.06344504552370865 2.188058792808226
-0.4280408678101212 4.822020295604495
-0.10271125813415245 2.4057507520488803
0.2254230517406202 3.5680157199458296
-0.14454501601841538 2.716829655180198
-0.5925020619766186 5.840333850120099
-0.252458105443717 3.560138438253747
0.07752659328477349 2.124956490585947
0.44951404559083513 6.017188065153747
0.1851404879665386 3.1408837090496804
-0.08631823483303416 2.30031815568744
-0.5747441808842243 5.737176880437012
-0.23550813414961813 3.430109797887234
0.09556123644771916 2.264986423956741
0.5157354282741491 6.7442431888135514
0.24991785187929277 3.8307469255888154
-0.02274101016886637 2.137258053940417
-0.312789659934541 4.014340136433345
0.010145829502572314 2.0
0.33346196398515865 4.740147271625051


KeyboardInterrupt: 